<a href="https://colab.research.google.com/github/ncpitrsb/Fetal-Health-with-XGBoost/blob/main/Fetal_Health_with_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Competition
(using RandomForest Model | Accuracy : ~90.82%)

In [ ]:
!wget https://raw.githubusercontent.com/Natthapolmnc/HealthClassification/main/dat/train.csv

--2021-02-18 15:21:23--  https://raw.githubusercontent.com/Natthapolmnc/HealthClassification/main/dat/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 378565 (370K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 369.69K  --.-KB/s    in 0.02s   

2021-02-18 15:21:24 (17.9 MB/s) - ‘train.csv.1’ saved [378565/378565]



In [ ]:
!wget https://raw.githubusercontent.com/Natthapolmnc/HealthClassification/main/dat/test.csv

--2021-02-18 15:21:24--  https://raw.githubusercontent.com/Natthapolmnc/HealthClassification/main/dat/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92446 (90K) [text/plain]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>]  90.28K  --.-KB/s    in 0.005s  

2021-02-18 15:21:24 (16.3 MB/s) - ‘test.csv.1’ saved [92446/92446]



# Data Preparing

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('train.csv')
data_x_test_for_sub = pd.read_csv('test.csv')

df = pd.DataFrame(data=data)



fetal_encode = {'Normal':1, 'Suspect':2, 'Pathological':3}
df["fetal_health"].replace(fetal_encode, inplace=True)

In [ ]:
df

In [ ]:
df.shape

(1700, 23)

In [ ]:
label = df[['fetal_health']]
df = df.drop(columns=["id","fetal_health"])

test_id = data_x_test_for_sub['id']
data_x_test_for_sub = data_x_test_for_sub.drop(columns=['id'])


In [ ]:
data_x_test_for_sub

**Split Data**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.25, random_state=42)

In [ ]:
X_train.shape

(1275, 21)

# **Generate Profiling (using Pandas-Profiling)**

In [ ]:
!pip install pandas-profiling==2.10.1

In [ ]:

from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="fetal_health_report")
profile.to_file("your_report.html")

Summarize dataset:   0%|          | 0/36 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile.to_file("your_report.html")

# Data Visualization

In [ ]:
import matplotlib.pyplot as plt
X_train.hist(figsize=(20,15))
plt.show()

In [ ]:
df_corr = X_train.corr(method='pearson')
df_corr

In [ ]:
import seaborn as sns 
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(df_corr, ax = ax, cmap ="rocket_r", linewidths = 0.1) 

In [ ]:
temp_df = X_train
for i in df_corr.columns:
  temp = [x for x in df_corr.columns if df_corr[i][x] > 0.75 and i != x]
  if len(temp) > 0:
    print(i,':', temp)
temp_df = temp_df.drop(columns=['histogram_median', 'histogram_mode', 'histogram_number_of_peaks'])
df_corr_2 = temp_df.corr(method='pearson')
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(df_corr_2, ax = ax, cmap ="rocket_r", linewidths = 0.1) 

In [ ]:
for i in df_corr_2.columns:
  temp = [x for x in df_corr_2.columns if df_corr_2[i][x] > 0.75 and i != x]
  if len(temp) > 0:
    print(i,':', temp)

baseline value : ['histogram_mean']
histogram_mean : ['baseline value']


In [ ]:
X_train = X_train.drop(columns=['histogram_median', 'histogram_mode', 'histogram_number_of_peaks'])
X_test = X_test.drop(columns=['histogram_median', 'histogram_mode', 'histogram_number_of_peaks'])
data_x_test_for_sub = data_x_test_for_sub.drop(columns=['histogram_median', 'histogram_mode', 'histogram_number_of_peaks'])

In [ ]:
X_train.head(10)

In [ ]:
X_train

In [ ]:
X_train.shape

(1275, 18)

In [ ]:
X_test.shape

(425, 18)

In [ ]:
y_test.shape

(425, 1)

# Evaluate Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
folds = 10
param_comb = 10
n_estimators = [100, 200, 300, 400, 500]
learning_rate = [0.05,0.1,0.15,0.2,0.25]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1001)
xgb = XGBClassifier()#random_state=42,learning_rate=0.1
random_search = RandomizedSearchCV(xgb, param_distributions=param_grid, n_iter=param_comb,cv=kfold,verbose=1,)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
# xgb.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.8min finished



 Time taken: 0 hours 1 minutes and 52.62 seconds.


In [ ]:
# print('♥ Train ♥')
# print(classification_report(y_train, random_search.predict(X_train)))
# # print("Accuracy:",metrics.accuracy_score(y_train, random_search.predict(X_train)))

print('♥ Val ♥')
print(classification_report(y_test, random_search.predict(X_test)))
# print("Accuracy:",metrics.accuracy_score(y_test, random_search.predict(X_test)))

In [ ]:
print("Confusion Matrix:")
y_pred = random_search.predict(X_test)
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[337   6   2]
 [  9  37   0]
 [  3   0  31]]


In [ ]:
random_search.best_params_

In [ ]:
X_train.shape

(1275, 18)

In [ ]:
data_x_test_for_sub.shape

(426, 18)

In [ ]:
y_pred_sub = random_search.predict(data_x_test_for_sub)
fetal_decode = {1:'Normal', 2:'Suspect', 3:'Pathological'}
y_pred_decode = [fetal_decode[x] for x in y_pred_sub]
d = {'id': test_id, 'fetal_health': y_pred_decode}
df_sub = pd.DataFrame(data=d)
df_sub

,id,fetal_health
0,0,Suspect
1,1,Normal
2,2,Normal
3,3,Normal
4,4,Normal
...,...,...
421,421,Normal
422,422,Normal
423,423,Normal
424,424,Suspect


In [ ]:
df_sub.to_csv('submission.csv', index = False)
from google.colab import files

files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>